<a href="https://colab.research.google.com/github/Wonderofyou/RAG-application/blob/main/RAG_application.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pymilvus  #This will prevent pymilvus from installing its own dependencies
!pip install peft
!pip install -U FlagEmbedding

In [2]:
import json
import copy

In [3]:
with open('articles.json', 'r') as f:
    data = json.load(f)

In [4]:
text = ""
for page in data:
  text = text + "Thể loại: "+ page['category']+'\n'
  text = text + "Ngày đăng: "+page['datetime']+'\n'
  text += page['content']
  text += "\n\n"
len(text)

4831879

In [5]:
!pip install langchain_text_splitter
!pip install langchain_google_genai
!pip install langchain_community

ERROR: Could not find a version that satisfies the requirement langchain_text_splitter (from versions: none)
ERROR: No matching distribution found for langchain_text_splitter


In [6]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1800, chunk_overlap=200, add_start_index=True
)

SPLITTING TEXT

In [7]:
all_splits = text_splitter.split_text(text = text)
all_splits[0], all_splits[1]

('Thể loại: thế giới\nNgày đăng: 2024-04-01T00:00:00\nValery Chaika, quan chức do Moskva hậu thuẫn ở Lugansk, thiệt mạng vì nổ bom xe, trong bối cảnh nhiều chính trị gia thân Nga bị ám sát thời gian qua.\nChi nhánh địa phương của Ủy ban Điều tra Nga cho biết phó giám đốc một cơ quan hành chính nhà nước đã thiệt mạng khi "thiết bị không xác định phát nổ trong ôtô" của ông vào chiều 1/4.\nỦy ban đăng bức ảnh chụp một chiếc SUV màu sáng với cửa sổ và cửa ra vào bị nổ tung, các mảnh vỡ vương vãi khắp con đường ở Starobilsk, thành phố thuộc tỉnh Lugansk.\n"Các tình tiết của vụ án và những người liên quan đến hành vi tội ác đang được xác định", cơ quan này cho biết, thêm rằng họ đã mở cuộc điều tra về "hành vi khủng bố".\nNgười đứng đầu thành phố Vladimir Chernev cho hay nạn nhân là Valery Chaika, một quan chức ở Lugansk.\n"Đồng nghiệp của chúng tôi đã chết", ông viết trong một bài đăng trên Telegram, kêu gọi người dân chú ý và báo cáo mọi hoạt động đáng ngờ cho chính quyền.\nLugansk là một 

LOADING MODEL GEMINI

In [8]:
from langchain_google_genai import ChatGoogleGenerativeAI
import os
from google.colab import userdata

GG_API_KEY = userdata.get("GOOGLE_API_KEY")

llm = ChatGoogleGenerativeAI(model="gemini-1.5-pro",google_api_key=GG_API_KEY)
llm

ChatGoogleGenerativeAI(model='models/gemini-1.5-pro', google_api_key=SecretStr('**********'), client=<google.ai.generativelanguage_v1beta.services.generative_service.client.GenerativeServiceClient object at 0x7d438064cbb0>, async_client=<google.ai.generativelanguage_v1beta.services.generative_service.async_client.GenerativeServiceAsyncClient object at 0x7d438064f640>, default_metadata=())

In [9]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001",google_api_key=GG_API_KEY)
vectors = embeddings.embed_documents(
    [
        "Today is Monday",
        "Today is Tuesday",
        "Today is April Fools day",
    ]
)
print(len(vectors), len(vectors[0]))

3 768


In [10]:
ZILLIZ_CLOUD_URI = userdata.get("ZILLIZ_CLOUD")  # example: "https://in01-17f69c292d4a5sa.aws-us-west-2.vectordb.zillizcloud.com:19536"
ZILLIZ_CLOUD_API_KEY = userdata.get("ZILLIZ_APIKEY")

In [12]:
from langchain_community.vectorstores import Milvus
connection_args = {
    "uri": ZILLIZ_CLOUD_URI,
    "token": ZILLIZ_CLOUD_API_KEY,  # Use your API key
    "secure": True,
}

# Establish connection to the existing Milvus collection
vector_db = Milvus(
    connection_args=connection_args,
    collection_name="LangChainCollection" ,
    embedding_function=embeddings
)


In [13]:
def Query(query, db):
    return db.similarity_search(query,k=20)

RERANKER

In [14]:
from FlagEmbedding import FlagReranker
reranker = FlagReranker('BAAI/bge-reranker-large', use_fp16=True) # Setting use_fp16 to True speeds up computation with a slight performance degradation

score = reranker.compute_score(['query', 'passage'])
print(score)

scores = reranker.compute_score([['what is panda?', 'hi'], ['what is panda?', 'The giant panda (Ailuropoda melanoleuca), sometimes called a panda bear or simply panda, is a bear species endemic to China.']])
print(scores)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/443 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/279 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/801 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

[-1.51953125]
[-5.59765625, 5.76171875]


In [15]:
def get_top_doc(query, retrival, reranker):
    docs = retrival.similarity_search(query, k=20)
    scores = reranker.compute_score([[query, doc.page_content] for doc in docs])
    sorted_pairs = sorted(zip(scores, docs), key=lambda x: x[0], reverse=True)
    docs_sorted = [doc for _, doc in sorted_pairs]
    return docs_sorted


In [16]:
data = get_top_doc(query="Ai là tổng thống Nga Putin?", retrival=vector_db, reranker=reranker)
len(data)

20

In [17]:
data[:5]

[Document(metadata={'pk': 451726416302961524}, page_content='Thể loại: thế giới\nNgày đăng: 2024-04-13T00:00:00\nMoskva nói dự thảo hòa bình được Nga - Ukraine thảo luận vào đầu xung đột năm 2022 có thể là khởi đầu cho tiến trình đàm phán hiện nay.\nPhát ngôn viên Điện Kremlin Dmitry Peskov hôm 12/4 cho biết dự thảo hòa bình, đã được phái đoàn Nga - Ukraine thảo luận ở Istanbul, Thổ Nhĩ Kỳ, hồi tháng 3/2022, có thể là "cơ sở để bắt đầu đàm phán" giữa hai nước hiện nay. Tuy nhiên, ông Peskov cũng lưu ý các cuộc đàm phán vẫn phải tính tới "thực tế mới" sau hơn hai năm chiến sự.\n"Từ đó tới nay đã có nhiều thay đổi, nhiều khu vực mới đã được sáp nhập vào lãnh thổ chúng tôi", phát ngôn viên Điện Kremlin nói.\nHồi tháng 9/2022, Nga tuyên bố sáp nhập 4 tỉnh Ukraine gồm Kherson, Zaporizhzhia, Donetsk và Lugansk, bất chấp sự phản đối từ Kiev và đồng minh. Nga tuyên bố sẽ không bao giờ từ bỏ 4 tỉnh này trong bất cứ cuộc đàm phán nào với Ukraine.\nHồi tháng 6/2023, Tổng thống Nga Vladimir Putin 

In [18]:
from typing import List

from langchain_core.callbacks import CallbackManagerForRetrieverRun
from langchain_core.documents import Document
from langchain_core.retrievers import BaseRetriever


class Retriever_Reranker(BaseRetriever):
    """A toy retriever that contains the top k documents that contain the user query.

    This retriever only implements the sync method _get_relevant_documents.

    If the retriever were to involve file access or network access, it could benefit
    from a native async implementation of `_aget_relevant_documents`.

    As usual, with Runnables, there's a default async implementation that's provided
    that delegates to the sync implementation running on another thread.
    """

    documents: List[str]
    """List of documents to retrieve from."""
    k: int
    """Number of top results to return"""

    def _get_relevant_documents(
        self, query: str, *, run_manager: CallbackManagerForRetrieverRun
    ) -> List[Document]:
        matching_documents = get_top_doc(query, vector_db, reranker)[:self.k]
        return matching_documents

retriever = Retriever_Reranker(documents=all_splits, k=5)


In [19]:
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

contextualize_q_system_prompt = """Given a chat history and the latest user question \
which might reference context in the chat history, formulate a standalone question \
which can be understood without the chat history. Do NOT answer the question, \
just reformulate it if needed and otherwise return it as is."""
contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)
history_aware_retriever = create_history_aware_retriever(
    llm, retriever, contextualize_q_prompt
)

In [20]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain

qa_system_prompt = """You are an assistant for question-answering tasks. \
Use the following pieces of retrieved context to answer the question. \
If you don't know the answer, just say that you don't know. \
Use three sentences maximum and keep the answer concise.\

{context}"""
qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", qa_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)


question_answer_chain = create_stuff_documents_chain(llm, qa_prompt)

rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)

In [21]:
from langchain_core.messages import HumanMessage

chat_history = []

question = "What is Task Decomposition?"
ai_msg_1 = rag_chain.invoke({"input": question, "chat_history": chat_history})
chat_history.extend([HumanMessage(content=question), ai_msg_1["answer"]])

second_question = "What are common ways of doing it?"
ai_msg_2 = rag_chain.invoke({"input": second_question, "chat_history": chat_history})

print(ai_msg_2["answer"])

This document does not contain the answer to the question "What are common ways of doing [task decomposition]?". This document focuses on Vietnamese regulations and business practices, and does not discuss concepts like task decomposition. 



In [ ]:
chat_history